In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup

In [ ]:
df_train = pd.read_csv('data/train.csv', sep='\t', encoding='utf8', index_col='id')
df_test = pd.read_csv('data/test.csv', sep='\t', encoding='utf8', index_col='id')
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train['description'] = df_train['description'].map(lambda s: BeautifulSoup(s, "lxml").get_text())
df_test['description'] = df_test['description'].map(lambda s: BeautifulSoup(s, "lxml").get_text())

In [ ]:
df_train['description'] = df_train['name'] + " " + df_train['description']
df_test['description'] = df_test['name'] + " " + df_test['description']

In [ ]:
df_train.head()

In [ ]:
df_train.to_csv('train_clean.csv', sep=',', encoding='utf8')
df_test.to_csv('test_clean.csv', sep=',', encoding='utf8')

In [2]:
#повторно запускать отсюда
df_train = pd.read_csv('train_clean.csv', sep=',', encoding='utf8', index_col='id')
df_test = pd.read_csv('test_clean.csv', sep=',', encoding='utf8', index_col='id')

In [3]:
df_test.head()

,name,description
id,,
200000,Дизайнер-консультант мебели,Дизайнер-консультант мебели Обязанности: Рабо...
200001,Продавец-консультант (ТЦ на Пушкина),Продавец-консультант (ТЦ на Пушкина) Обязаннос...
200002,Менеджер по продажам,Менеджер по продажам Торговый Дом «Форт» это в...
200003,Продавец-консультант в магазин одежды (ТЦ Волн...,Продавец-консультант в магазин одежды (ТЦ Волн...
200004,Специалист по охране труда,Специалист по охране труда Обязанности: осущ...


In [4]:
df_train.head()

,name,description,target
id,,,
0,Заведующий отделом/секцией в магазин YORK (Уру...,Заведующий отделом/секцией в магазин YORK (Уру...,1
1,Наладчик станков и манипуляторов с ПУ,Наладчик станков и манипуляторов с ПУ Обязанно...,0
2,Разработчик С++ (Криптограф),Разработчик С++ (Криптограф) Требования: Опыт...,0
3,Фрезеровщик,Фрезеровщик Условия: На работу вахтовым метод...,0
4,Мерчендайзер/продавец-консультант,Мерчендайзер/продавец-консультант Компания Пал...,1


In [5]:
X_text = []
for i in df_train['description']:
       X_text.append(i)
y = df_train['target'].values

In [6]:
X_text_test = []
for i in df_test['description']:
       X_text_test.append(i)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size=0.25, shuffle=True, random_state=123)

In [ ]:
# попробуем разные модели

In [8]:
# tfidf + rfc (Roc_auc = 0.9911)
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,1))),
    ('rfc', RandomForestClassifier(n_estimators = 100, n_jobs=-1)),
])

model.fit(X_train, y_train)
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

0.9909794785153723

In [7]:
# tfidf + rfc 2 (работает ОЧЕНЬ! долго Roc_auc = 0.990673)
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))), #учитываем словосочетания
    ('rfc', RandomForestClassifier(n_estimators = 100, n_jobs=-1)),
])

model.fit(X_train, y_train)
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

0.9906734958864986

In [9]:
#countvect +lr (Roc_auc = 0.989412)
model_2 = Pipeline([
    ('cnt_vect', CountVectorizer(ngram_range=(1,1))),
    ('lr', LogisticRegression(random_state=123)),
])

model_2.fit(X_train, y_train)
roc_auc_score(y_test, model_2.predict_proba(X_test)[:,1])

0.9894127986079114

In [10]:
#tfidf + lr (Roc_auc = 0.9905572)
model_3 = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,1))),
    ('lr', LogisticRegression(random_state=123)),
])

model_3.fit(X_train, y_train)
roc_auc_score(y_test, model_3.predict_proba(X_test)[:,1])

0.990557254452333

In [11]:
# tfid + dtc (Roc_auc = 0.9392004)
model_4 = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,1))),
    ('dtc', DecisionTreeClassifier()),
])

model_4.fit(X_train, y_train)
roc_auc_score(y_test, model_4.predict_proba(X_test)[:,1])

0.9392004279219345

In [10]:
counter = TfidfVectorizer(ngram_range=(1,1))
#counter = CountVectorizer(ngram_range=(1,1))
X = counter.fit_transform(X_text)
X_valid = counter.transform(X_text_test)

In [43]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

In [11]:
models = [
    RandomForestClassifier(n_jobs=-1, random_state=123),
    LogisticRegression(random_state=123),
   # KNeighborsClassifier(n_neighbors=11),
   # GaussianNB()
]

In [12]:
def roc_auc_score_cust(y_true, y_hat):
    return roc_auc_score(y_true, y_hat[:,1])

In [13]:
# пробуем стэкинг
from vecstack import stacking

S_train, S_valid = stacking(models,
                               X, y, X_valid,
                               regression=False,
                               mode='oof_pred_bag', 
                               needs_proba=True,
                               metric=roc_auc_score_cust,
                               n_folds=5,                
                               stratified=True,          
                               shuffle=True,             
                               random_state=123,         
                               verbose=2)



task:         [classification]
n_classes:    [2]
metric:       [roc_auc_score_cust]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [0.98524359]
    fold  1:  [0.98563448]
    fold  2:  [0.98598861]
    fold  3:  [0.98617603]
    fold  4:  [0.98612142]
    ----
    MEAN:     [0.98583283] + [0.00034982]
    FULL:     [0.98582920]

model  1:     [LogisticRegression]
    fold  0:  [0.99035889]
    fold  1:  [0.99056848]
    fold  2:  [0.99113602]
    fold  3:  [0.99082014]
    fold  4:  [0.99110929]
    ----
    MEAN:     [0.99079856] + [0.00030237]
    FULL:     [0.99079745]



In [14]:
#(Roc_auc = 0.99240)
last_model = GradientBoostingClassifier(n_estimators=300, max_depth=4,
                                       learning_rate=0.01,
                                       random_state=123)
last_model.fit(S_train, y)
y_hat = last_model.predict_proba(S_valid)

roc_auc_score(y, last_model.predict_proba(S_train)[:,1])

0.9924040544514277

In [55]:
# запись результатов в файл
df_results=df_test.reset_index()
df_results = df_results.drop(['name', 'description'], axis = 1)

In [56]:
df_results = pd.concat([df_results,pd.DataFrame(y_hat[:, 1] ,columns = ['target'])],axis = 1, names = ['id','target'])
df_results.head()

,id,target
0,200000,0.981509
1,200001,0.998956
2,200002,0.993868
3,200003,0.998698
4,200004,0.001150


In [57]:
df_results.to_csv('submission.csv', sep=',', encoding='utf8',index = False)